In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime
#from kaggle.competitions import nflrush
import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
import keras
from tqdm import tqdm_notebook

# Options
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000
pd.set_option('display.max_columns', None)
import gc
gc.enable()

sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [15,10]

Using TensorFlow backend.


In [2]:
#train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv', dtype={'WindSpeed': 'object'})
train = pd.read_csv('../data/train.csv', dtype={'WindSpeed': 'object'}, low_memory=False)
# train = train[:2200]
print(train.shape)
train.head()

(509762, 49)


,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


## Feature engineering

In [3]:
#https://www.kaggle.com/rooshroosh/fork-of-neural-networks-different-architecture
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

def strtofloat(x):
    try:
        return float(x)
    except:
        return -1

def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def orientation_to_cat(x):
    x = np.clip(x, 0, 360 - 1)
    try:
        return str(int(x/15))
    except:
        return "nan"
    
# Funcao para agrupar as descricoes dos tipos de estadio
def agrupar_tipo_estadio(StadiumType):
    outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
    indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
    indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
    dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
    dome_open     = ['Domed, Open', 'Domed, open']
    
    if StadiumType in outdoor:
        return 'outdoor'
    elif StadiumType in indoor_closed:
        return 'indoor_closed'
    elif StadiumType in indoor_open:
        return 'indoor_open'
    elif StadiumType in dome_closed:
        return 'dome_closed'
    elif StadiumType in dome_open:
        return 'dome_open'
    else:
        return 'outdoor' # se for n/a
    
# Funcao para agrupar as descricoes dos estadios
def agrupar_estadio(Stadium):
    if Stadium == 'Broncos Stadium at Mile High':
        return 'Broncos Stadium At Mile High'
    elif Stadium in ('CenturyField', 'CenturyLink'):
        return 'CenturyLink Field'
    elif Stadium == 'EverBank Field':
        return 'Everbank Field'
    elif Stadium in ('FirstEnergy', 'FirstEnergy Stadium', 'FirstEnergyStadium'):
        return 'First Energy Stadium'
    elif Stadium == 'Lambeau field':
        return 'Lambeau Field'
    elif Stadium == 'Los Angeles Memorial Coliesum':
        return 'Los Angeles Memorial Coliseum'
    elif Stadium in ('M & T Bank Stadium', 'M&T Stadium'):
        return 'M&T Bank Stadium'
    elif Stadium in ('Mercedes-Benz Dome', 'Mercedes-Benz Superdome'):
        return 'Mercedes-Benz SuperDome'
    elif Stadium in ('MetLife Stadium', 'Metlife Stadium', 'MetLife'):
        return 'MetLife Stadium' 
    elif Stadium == 'NRG':
        return 'NRG Stadium' 
    elif Stadium == 'Oakland-Alameda County Coliseum':
        return 'Oakland Alameda-County Coliseum' 
    elif Stadium == 'Paul Brown Stdium':
        return 'Paul Brown Stadium' 
    elif Stadium == 'Twickenham':
        return 'Twickenham Stadium' 
    else:
        return Stadium
    
# Funcao para agrupar a localizacao do estadio e do jogo
def agrupar_local(Location):

    if Location == "Arlington, Texas":
        return "Arlington, TX"
    elif Location in ("Baltimore, Maryland","Baltimore, Md."):
        return "Baltimore, MD"
    elif Location == "Charlotte, North Carolina":
        return "Charlotte, NC"
    elif Location == "Chicago. IL":
        return "Chicago, IL"
    elif Location == "Cincinnati, Ohio":
        return "Cincinnati, OH"
    elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
        return "Cleveland, OH"
    elif Location == "Detroit":
        return "Detroit, MI"
    elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
        return "East Rutherford, NJ"
    elif Location == "Foxborough, Ma":
        return "Foxborough, MA"
    elif Location == "Houston, Texas":
        return "Houston, TX"
    elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
        return "Jacksonville, FL"
    elif Location == "London":
        return "London, England"
    elif Location == "Los Angeles, Calif.":
        return "Los Angeles, CA"
    elif Location == "Miami Gardens, Fla.":
        return "Miami Gardens, FLA"
    elif Location in ("New Orleans","New Orleans, La."):
        return "New Orleans, LA"
    elif Location == "Orchard Park NY":
        return "Orchard Park, NY"
    elif Location == "Philadelphia, Pa.":
        return "Philadelphia, PA"
    elif Location == "Pittsburgh":
        return "Pittsburgh, PA"
    elif Location == "Seattle":
        return "Seattle, WA"
    else:
        return Location
    

# Funcao para agrupar os dados de direcao do vento
def agrupa_wind_direction(WindDirection):
    wd = str(WindDirection).upper()
    
    if wd == 'N' or 'FROM N' in wd:
        return 'north'
    if wd == 'S' or 'FROM S' in wd:
        return 'south'
    if wd == 'W' or 'FROM W' in wd:
        return 'west'
    if wd == 'E' or 'FROM E' in wd:
        return 'east'
    
    if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
        return 'south west'
    if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
        return 'south east'
    if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
        return 'north west'
    if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
        return 'north east'
    
    if 'NW' in wd or 'NORTHWEST' in wd:
        return 'north west'
    if 'NE' in wd or 'NORTH EAST' in wd:
        return 'north east'
    if 'SW' in wd or 'SOUTHWEST' in wd:
        return 'south west'
    if 'SE' in wd or 'SOUTHEAST' in wd:
        return 'south east'

    return 'unknown'

    
# Funcao para converter a velocidade do vento
def convert_wind_speed(WindSpeed):
    ws = str(WindSpeed)

    if ws.isdigit():
        return int(ws)

    if '-' in ws:
        return int(ws.split('-')[0])

    if ws.split(' ')[0].isdigit():
        return int(ws.split(' ')[0])

    if 'mph' in ws.lower():
        return int(ws.lower().split('mph')[0])
    else:
        return 0

def transform_time_quarter(str1):
    return int(str1[:2])*60 + int(str1[3:5])

def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60

In [4]:
train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [5]:
# Funcao para realizar feature engineering no dataset treino
def features_estatisticas(data):

    df = pd.DataFrame()
    
    for col in data.columns:
        if col in ['Yards','GameId','PlayId','Team','DisplayName','Season','GameClock','PossessionTeam','FieldPosition', 
                   'NflIdRusher', 'OffenseFormation', 'OffensePersonnel', 'DefensePersonnel',
                   'PlayDirection', 'TimeHandoff', 'TimeSnap', 'PlayerHeight', 'PlayerBirthDate', 'PlayerCollegeName', 'Position',
                   'HomeTeamAbbr', 'VisitorTeamAbbr', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
                   'WindSpeed','WindDirection']:
            continue
        print(col)  
        df[col + '_mean'] = data.groupby(['PlayId'])[col].mean()
        df[col + '_median'] = data.groupby(['PlayId'])[col].median()
        #df[col + '_max'] = data.groupby(['PlayId'])[col].max()
        #df[col + '_min'] = data.groupby(['PlayId'])[col].min()
        df[col + '_std'] = data.groupby(['PlayId'])[col].std()
        #df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        #df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        #df[col + '_mean_abs_chg'] = data.groupby(['PlayId'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        #df[col + '_mean_change_of_abs_change'] = data.groupby(['PlayId'])[col].apply(mean_change_of_abs_change)
        #df[col + '_abs_max'] = data.groupby(['PlayId'])[col].apply(lambda x: np.max(np.abs(x)))
        #df[col + '_abs_min'] = data.groupby(['PlayId'])[col].apply(lambda x: np.min(np.abs(x)))
        #df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2

        # Advanced Features
        #df[col + '_skew'] = data.groupby(['PlayId'])[col].skew()
        #df[col + '_mad'] = data.groupby(['PlayId'])[col].mad()
        #df[col + '_q25'] = data.groupby(['PlayId'])[col].quantile(0.25)
        #df[col + '_q75'] = data.groupby(['PlayId'])[col].quantile(0.75)
        #df[col + '_q95'] = data.groupby(['PlayId'])[col].quantile(0.95)
        #df[col + '_iqr'] = df[col + '_q75'] - df[col + '_q25']
        #df[col + '_SSC'] = data.groupby(['PlayId'])[col].apply(SSC) 
        #df[col + '_skewness'] = data.groupby(['PlayId'])[col].apply(skewness)
        #df[col + '_wave_lenght'] = data.groupby(['PlayId'])[col].apply(wave_length)
        #df[col + '_norm_entropy'] = data.groupby(['PlayId'])[col].apply(norm_entropy)
        #df[col + '_SRAV'] = data.groupby(['PlayId'])[col].apply(SRAV)
        #df[col + '_kurtosis'] = data.groupby(['PlayId'])[col].apply(_kurtosis) 
        #df[col + '_zero_crossing'] = data.groupby(['PlayId'])[col].apply(zero_crossing) 

    return df
    

In [6]:
# Criando um novo dataset aplicando Feature Engineering
data = features_estatisticas(train)

X
Y
S
A
Dis
Orientation
Dir
NflId
JerseyNumber
YardLine
Quarter
Down
Distance
HomeScoreBeforePlay
VisitorScoreBeforePlay
DefendersInTheBox
PlayerWeight
Week
Temperature
Humidity


In [7]:
data.head()

,X_mean,X_median,X_std,Y_mean,Y_median,Y_std,S_mean,S_median,S_std,A_mean,A_median,A_std,Dis_mean,Dis_median,Dis_std,Orientation_mean,Orientation_median,Orientation_std,Dir_mean,Dir_median,Dir_std,NflId_mean,NflId_median,NflId_std,JerseyNumber_mean,JerseyNumber_median,JerseyNumber_std,YardLine_mean,YardLine_median,YardLine_std,Quarter_mean,Quarter_median,Quarter_std,Down_mean,Down_median,Down_std,Distance_mean,Distance_median,Distance_std,HomeScoreBeforePlay_mean,HomeScoreBeforePlay_median,HomeScoreBeforePlay_std,VisitorScoreBeforePlay_mean,VisitorScoreBeforePlay_median,VisitorScoreBeforePlay_std,DefendersInTheBox_mean,DefendersInTheBox_median,DefendersInTheBox_std,PlayerWeight_mean,PlayerWeight_median,PlayerWeight_std,Week_mean,Week_median,Week_std,Temperature_mean,Temperature_median,Temperature_std,Humidity_mean,Humidity_median,Humidity_std
PlayId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20170907000118,73.881818,74.590,4.392297,28.713182,29.305,6.308114,1.710455,1.440,1.258432,1.191818,0.980,0.691807,0.245000,0.205,0.184462,188.772273,178.895,113.044188,207.868182,209.315,69.109720,2.231286e+06,2537144.0,777697.534330,50.818182,52.5,26.433450,35,35,0.0,1,1,0.0,3,3,0.0,2,2,0.0,0,0,0.0,0,0,0.0,6.0,6.0,0.0,246.363636,235.0,51.293317,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0
20170907000139,65.785455,66.530,4.647315,24.827727,27.340,6.892898,1.866818,1.745,0.830573,1.567273,1.645,0.617554,0.236364,0.230,0.121088,185.461364,175.330,115.606582,193.386818,236.235,110.152689,2.231286e+06,2537144.0,777697.534330,50.818182,52.5,26.433450,43,43,0.0,1,1,0.0,1,1,0.0,10,10,0.0,0,0,0.0,0,0,0.0,6.0,6.0,0.0,246.363636,235.0,51.293317,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0
20170907000189,43.381364,44.025,4.003402,22.026818,22.220,6.192438,3.463636,3.485,0.986512,1.755909,1.410,0.935733,0.380455,0.395,0.103853,124.238182,125.765,108.112184,202.421364,197.535,36.419174,2.231286e+06,2537144.0,777697.534330,50.818182,52.5,26.433450,35,35,0.0,1,1,0.0,1,1,0.0,10,10,0.0,0,0,0.0,0,0,0.0,7.0,7.0,0.0,246.363636,235.0,51.293317,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0
20170907000345,11.485000,11.230,1.679696,25.766818,25.965,3.795418,1.848636,1.575,1.171463,1.087273,0.840,0.825499,0.226818,0.190,0.116062,166.748182,182.395,122.357177,240.664091,306.995,125.543391,2.231050e+06,2519447.5,765789.830465,61.318182,60.5,25.400387,2,2,0.0,1,1,0.0,2,2,0.0,2,2,0.0,0,0,0.0,0,0,0.0,9.0,9.0,0.0,270.000000,267.5,42.890780,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0
20170907000395,35.774545,34.290,4.811638,26.824091,26.955,7.466987,2.483636,2.600,0.799247,1.870909,1.780,0.627489,0.288182,0.315,0.125381,138.934091,150.145,110.245595,170.335000,175.330,76.029425,2.336007e+06,2542471.0,667939.348952,55.590909,57.5,29.129692,25,25,0.0,1,1,0.0,1,1,0.0,10,10,0.0,7,7,0.0,0,0,0.0,7.0,7.0,0.0,255.454545,249.0,51.232177,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0


In [8]:
def preprocess(train):
    
    train.loc[train['PlayDirection'] == 'left', 'X'] = 120 - train.loc[train['PlayDirection'] == 'left', 'X']
    train.loc[train['PlayDirection'] == 'left', 'Y'] = (160 / 3) - train.loc[train['PlayDirection'] == 'left', 'Y']
    train.loc[train['PlayDirection'] == 'left', 'Orientation'] = np.mod(180 + train.loc[train['PlayDirection'] == 'left', 'Orientation'], 360)
    train.loc[train['PlayDirection'] == 'left', 'Dir'] = np.mod(180 + train.loc[train['PlayDirection'] == 'left', 'Dir'], 360)
    train['FieldPosition'].fillna('', inplace=True)
    train.loc[train['PossessionTeam'] != train['FieldPosition'], 'YardLine'] = 100 - train.loc[train['PossessionTeam'] != train['FieldPosition'], 'YardLine']
    #train["Season"] = train[train["Season"] != 2017]

    ## GameClock
    train['GameClock_sec'] = train['GameClock'].apply(strtoseconds)
    train["GameClock_minute"] = train["GameClock"].apply(lambda x : x.split(":")[0]).astype("object")

    ## Height
    train['PlayerHeight_dense'] = train['PlayerHeight'].apply(lambda x: (12*int(x.split('-')[0])+int(x.split('-')[1]) * 2.54/100))

    ## Time
    train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

    train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

    ## Age
    seconds_in_year = 60*60*24*365.25
    train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    train["PlayerAge_ob"] = train['PlayerAge'].astype(np.int).astype("object")

    ## WindSpeed
    train['WindSpeed_ob'] = train['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    train['WindSpeed_ob'] = train['WindSpeed_ob'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    train['WindSpeed_ob'] = train['WindSpeed_ob'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    train['WindSpeed_dense'] = train['WindSpeed_ob'].apply(strtofloat)

    ## Weather
    train['GameWeather_process'] = train['GameWeather'].str.lower()
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: "indoor" if not pd.isna(x) and "indoor" in x else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    train['GameWeather_dense'] = train['GameWeather_process'].apply(map_weather)

    ## Rusher
    train['IsRusher'] = (train['NflId'] == train['NflIdRusher'])
    train['IsRusher_ob'] = (train['NflId'] == train['NflIdRusher']).astype("object")
    temp = train[train["IsRusher"]][["Team", "PlayId"]].rename(columns={"Team":"RusherTeam"})
    train = train.merge(temp, on = "PlayId")
    train["IsRusherTeam"] = train["Team"] == train["RusherTeam"]

    ## dense -> categorical
    train["Quarter_ob"] = train["Quarter"].astype("object")
    train["Down_ob"] = train["Down"].astype("object")
    train["JerseyNumber_ob"] = train["JerseyNumber"].astype("object")
    train["YardLine_ob"] = train["YardLine"].astype("object")
    train["DefendersInTheBox_ob"] = train["DefendersInTheBox"].astype("object")
    train["Week_ob"] = train["Week"].astype("object")
    train["TimeDelta_ob"] = train["TimeDelta"].astype("object")


    ## Orientation and Dir
    train["Orientation_ob"] = train["Orientation"].apply(lambda x : orientation_to_cat(x)).astype("object")
    train["Dir_ob"] = train["Dir"].apply(lambda x : orientation_to_cat(x)).astype("object")

    train["Orientation_sin"] = train["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    train["Orientation_cos"] = train["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    train["Dir_sin"] = train["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    train["Dir_cos"] = train["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

    
    # Runner horizontal speed
    radian_angle = (90 - train['Dir']) * np.pi / 180.0
    train['v_horizontal'] = np.abs(train['S'] * np.cos(radian_angle))
    train['v_vertical'] = np.abs(train['S'] * np.sin(radian_angle))


    ## diff Score
    train["diffScoreBeforePlay"] = train["HomeScoreBeforePlay"] - train["VisitorScoreBeforePlay"]
    train["diffScoreBeforePlay_binary_ob"] = (train["HomeScoreBeforePlay"] > train["VisitorScoreBeforePlay"]).astype("object")

    ## Turf
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    train['Turf'] = train['Turf'].map(Turf)

    ## OffensePersonnel
    temp = train["OffensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(OffensePersonnelSplit(x)))
    temp.columns = ["Offense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = "PlayId")

    ## DefensePersonnel
    temp = train["DefensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(DefensePersonnelSplit(x)))
    temp.columns = ["Defense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = "PlayId")

    # Rodrigo
    train.loc[train['Season'] == 2017, 'Orientation'] = np.mod(90 + train.loc[train['Season'] == 2017, 'Orientation'], 360)
    train['StadiumType'] = train['StadiumType'].apply(agrupar_tipo_estadio)
    train['Stadium'] = train['Stadium'].apply(agrupar_estadio)
    train['Location'] = train['Location'].apply(agrupar_local)
    train.loc[train.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    train.loc[train.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"
    train.loc[train.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    train.loc[train.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"
    train.loc[train.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    train.loc[train.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"
    train.loc[train.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    train.loc[train.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"   
    train['time_quarter'] = train.GameClock.map(lambda x:transform_time_quarter(x))
    train['time_end'] = train.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
    


    ## sort
#     train = train.sort_values(by = ['X']).sort_values(by = ['Dis']).sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index(drop = True)
    train = train.sort_values(by = ['X']).sort_values(by = ['Dis']).sort_values(by=['PlayId', 'IsRusherTeam', 'IsRusher']).reset_index(drop = True)
    return train

In [9]:
%%time
train = preprocess(train)

CPU times: user 2min 40s, sys: 6.23 s, total: 2min 46s
Wall time: 2min 25s


In [10]:
train = pd.merge(data, train, on='PlayId', how='outer')

In [11]:
## DisplayName remove Outlier
v = train["DisplayName"].value_counts()
missing_values = list(v[v < 5].index)
train["DisplayName"] = train["DisplayName"].where(~train["DisplayName"].isin(missing_values), "nan")

## PlayerCollegeName remove Outlier
v = train["PlayerCollegeName"].value_counts()
missing_values = list(v[v < 10].index)
train["PlayerCollegeName"] = train["PlayerCollegeName"].where(~train["PlayerCollegeName"].isin(missing_values), "nan")


In [12]:
train.head()

,PlayId,X_mean,X_median,X_std,Y_mean,Y_median,Y_std,S_mean,S_median,S_std,A_mean,A_median,A_std,Dis_mean,Dis_median,Dis_std,Orientation_mean,Orientation_median,Orientation_std,Dir_mean,Dir_median,Dir_std,NflId_mean,NflId_median,NflId_std,JerseyNumber_mean,JerseyNumber_median,JerseyNumber_std,YardLine_mean,YardLine_median,YardLine_std,Quarter_mean,Quarter_median,Quarter_std,Down_mean,Down_median,Down_std,Distance_mean,Distance_median,Distance_std,HomeScoreBeforePlay_mean,HomeScoreBeforePlay_median,HomeScoreBeforePlay_std,VisitorScoreBeforePlay_mean,VisitorScoreBeforePlay_median,VisitorScoreBeforePlay_std,DefendersInTheBox_mean,DefendersInTheBox_median,DefendersInTheBox_std,PlayerWeight_mean,PlayerWeight_median,PlayerWeight_std,Week_mean,Week_median,Week_std,Temperature_mean,Temperature_median,Temperature_std,Humidity_mean,Humidity_median,Humidity_std,GameId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,GameClock_sec,GameClock_minute,PlayerHeight_dense,TimeDelta,PlayerAge,PlayerAge_ob,WindSpeed_ob,WindSpeed_dense,GameWeather_process,GameWeather_dense,IsRusher,IsRusher_ob,RusherTeam,IsRusherTeam,Quarter_ob,Down_ob,JerseyNumber_ob,YardLine_ob,DefendersInTheBox_ob,Week_ob,TimeDelta_ob,Orientation_ob,Dir_ob,Orientation_sin,Orientation_cos,Dir_sin,Dir_cos,v_horizontal,v_vertical,diffScoreBeforePlay,diffScoreBeforePlay_binary_ob,OffenseDB,OffenseDL,OffenseLB,OffenseOL,OffenseQB,OffenseRB,OffenseTE,OffenseWR,DefenseDB,DefenseDL,DefenseLB,DefenseOL,time_quarter,time_end
0,20170907000118,73.881818,74.59,4.392297,28.713182,29.305,6.308114,1.710455,1.44,1.258432,1.191818,0.98,0.691807,0.245,0.205,0.184462,188.772273,178.895,113.044188,207.868182,209.315,69.10972,2.231286e+06,2537144.0,777697.53433,50.818182,52.5,26.43345,35,35,0.0,1,1,0.0,3,3,0.0,2,2,0.0,0,0,0.0,0,0,0.0,6.0,6.0,0.0,246.363636,235.0,51.293317,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0,2017090700,away,45.85,24.433333,0.72,0.73,0.01,252.58,94.14,2556369,Chris Jones,95,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,6-6,308,1994-07-03,Mississippi State,DT,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Artificial,Clear and warm,63.0,77.0,8,SW,854.0,14,72.1524,1.0,23.184204,23,8,8.0,clear and warm,1.0,False,False,home,False,1,3,95,35,6,1,1,10,6,0.299374,-0.954136,0.997391,-0.072194,0.718121,0.051980,0,False,0,0,0,0,0,1,1,3,6,2,3,0,854,46
1,20170907000118,73.881818,74.59,4.392297,28.713182,29.305,6.308114,1.710455,1.44,1.258432,1.191818,0.98,0.691807,0.245,0.205,0.184462,188.772273,178.895,113.044188,207.868182,209.315,69.10972,2.231286e+06,2537144.0,777697.53433,50.818182,52.5,26.43345,35,35,0.0,1,1,0.0,3,3,0.0,2,2,0.0,0,0,0.0,0,0,0.0,6.0,6.0,0.0,246.363636,235.0,51.293317,1,1,0.0,63.0,63.0,0.0,77.0,77.0,0.0,2017090700,away,45.33,20.693333,0.42,1.35,0.01,297.61,18.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,6-3,288,1989-03-25,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Artificial,Clear and warm,63.0,77.0,8,SW,854.0,14,72.0762,1.0,28.457305,28,8,8.0,clear and warm,1.0,False,False,home,False,1,3,97,35,6,1,1,13,1,-0.463451,-0.886123,0.320613,0.947210,0.134657,0.397828,0,False,0,0,0,0,0,1,1,3,6,2,3,0,854,46
2,20170907000118,73.881818,74.59,4.392297,28.713182,29.305,6.308114,1.710455,1.44,1.258432,1.191818,0.98,0.691807,0.245,0.205,0.184462,188.772273,178.895,113.044188,207.868182,209.315,69.10972,2.231286e+06,2537144.0,

In [13]:
#pd.to_pickle(train, "train.pkl")

In [14]:
def drop(train):
    drop_cols = ['GameId', 'GameWeather', 'NflId', 'Season', 'NflIdRusher'] 
    drop_cols += ['TimeHandoff', 'TimeSnap', 'PlayerBirthDate']
    drop_cols += ['Orientation', "Dir", 'WindSpeed', 'GameClock', 'PlayDirection']
    # drop_cols += ["DefensePersonnel","OffensePersonnel"]
    train = train.drop(drop_cols, axis = 1)
    return train

In [15]:
train = drop(train)

In [16]:
cat_features = []
dense_features = []
for col in train.columns:
    if train[col].dtype =='object':
        cat_features.append(col)
        print("*cat*", col, len(train[col].unique()))
    else:
        dense_features.append(col)
        print("!dense!", col, len(train[col].unique()))
dense_features.remove("PlayId")
dense_features.remove("Yards")

!dense! PlayId 23171
!dense! X_mean 22598
!dense! X_median 14425
!dense! X_std 23171
!dense! Y_mean 18862
!dense! Y_median 3737
!dense! Y_std 23169
!dense! S_mean 12096
!dense! S_median 1149
!dense! S_std 23108
!dense! A_mean 8415
!dense! A_median 699
!dense! A_std 23069
!dense! Dis_mean 2523
!dense! Dis_median 140
!dense! Dis_std 19213
!dense! Orientation_mean 22573
!dense! Orientation_median 16678
!dense! Orientation_std 23171
!dense! Dir_mean 22563
!dense! Dir_median 17958
!dense! Dir_std 23171
!dense! NflId_mean 19851
!dense! NflId_median 4804
!dense! NflId_std 19906
!dense! JerseyNumber_mean 517
!dense! JerseyNumber_median 86
!dense! JerseyNumber_std 17639
!dense! YardLine_mean 50
!dense! YardLine_median 50
!dense! YardLine_std 1
!dense! Quarter_mean 5
!dense! Quarter_median 5
!dense! Quarter_std 1
!dense! Down_mean 4
!dense! Down_median 4
!dense! Down_std 1
!dense! Distance_mean 35
!dense! Distance_median 35
!dense! Distance_std 1
!dense! HomeScoreBeforePlay_mean 50
!dense! HomeS

## categorical

In [17]:
train_cat = train[cat_features]
categories = []
most_appear_each_categories = {}
for col in tqdm_notebook(train_cat.columns):
    train_cat.loc[:,col] = train_cat[col].fillna("nan")
    train_cat.loc[:,col] = col + "__" + train_cat[col].astype(str)
    most_appear_each_categories[col] = list(train_cat[col].value_counts().index)[0]
    categories.append(train_cat[col].unique())
categories = np.hstack(categories)
print(len(categories))


3239


In [18]:
#train_cat = pd.get_dummies(categories)

In [19]:
le = LabelEncoder()
le.fit(categories)
for col in tqdm_notebook(train_cat.columns):
    train_cat.loc[:, col] = le.transform(train_cat[col])
num_classes = len(le.classes_)

In [20]:
train_cat.head()

,Team,DisplayName,PossessionTeam,FieldPosition,OffenseFormation,OffensePersonnel,DefensePersonnel,PlayerHeight,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Stadium,Location,StadiumType,Turf,WindDirection,GameClock_minute,PlayerAge_ob,WindSpeed_ob,GameWeather_process,IsRusher_ob,RusherTeam,Quarter_ob,Down_ob,JerseyNumber_ob,YardLine_ob,DefendersInTheBox_ob,Week_ob,TimeDelta_ob,Orientation_ob,Dir_ob,diffScoreBeforePlay_binary_ob
0,2990,453,2929,2228,2470,2483,20,2880,2720,2889,2318,3016,2963,2442,2952,2999,3086,2254,2558,3131,2260,2330,2947,2941,2205,2426,3166,7,3033,2993,2532,70,3237
1,2990,142,2929,2228,2470,2483,20,2877,2707,2887,2318,3016,2963,2442,2952,2999,3086,2254,2563,3131,2260,2330,2947,2941,2205,2428,3166,7,3033,2993,2535,51,3237
2,2990,707,2929,2228,2470,2483,20,2877,2817,2894,2318,3016,2963,2442,2952,2999,3086,2254,2569,3131,2260,2330,2947,2941,2205,2383,3166,7,3033,2993,2533,61,3237
3,2990,1998,2929,2228,2470,2483,20,2869,2765,2885,2318,3016,2963,2442,2952,2999,3086,2254,2560,3131,2260,2330,2947,2941,2205,2364,3166,7,3033,2993,2533,73,3237
4,2990,1708,2929,2228,2470,2483,20,2874,2778,2885,2318,3016,2963,2442,2952,2999,3086,2254,2561,3131,2260,2330,2947,2941,2205,2347,3166,7,3033,2993,2553,70,3237


## Dense

In [21]:
train_dense = train[dense_features]
sss = {}
medians = {}
for col in tqdm_notebook(train_dense.columns):
    print(col)
    medians[col] = np.nanmedian(train_dense[col])
    train_dense.loc[:, col] = train_dense[col].fillna(medians[col])
    ss = StandardScaler()
    train_dense.loc[:, col] = ss.fit_transform(train_dense[col].values[:,None])
    sss[col] = ss

X_mean
X_median
X_std
Y_mean
Y_median
Y_std
S_mean
S_median
S_std
A_mean
A_median
A_std
Dis_mean
Dis_median
Dis_std
Orientation_mean
Orientation_median
Orientation_std
Dir_mean
Dir_median
Dir_std
NflId_mean
NflId_median
NflId_std
JerseyNumber_mean
JerseyNumber_median
JerseyNumber_std
YardLine_mean
YardLine_median
YardLine_std
Quarter_mean
Quarter_median
Quarter_std
Down_mean
Down_median
Down_std
Distance_mean
Distance_median
Distance_std
HomeScoreBeforePlay_mean
HomeScoreBeforePlay_median
HomeScoreBeforePlay_std
VisitorScoreBeforePlay_mean
VisitorScoreBeforePlay_median
VisitorScoreBeforePlay_std
DefendersInTheBox_mean
DefendersInTheBox_median
DefendersInTheBox_std
PlayerWeight_mean
PlayerWeight_median
PlayerWeight_std
Week_mean
Week_median
Week_std
Temperature_mean
Temperature_median
Temperature_std
Humidity_mean
Humidity_median
Humidity_std
X
Y
S
A
Dis
JerseyNumber
YardLine
Quarter
Down
Distance
HomeScoreBeforePlay
VisitorScoreBeforePlay
DefendersInTheBox
PlayerWeight
Week
Temperature

In [22]:
train_dense.head()

,X_mean,X_median,X_std,Y_mean,Y_median,Y_std,S_mean,S_median,S_std,A_mean,A_median,A_std,Dis_mean,Dis_median,Dis_std,Orientation_mean,Orientation_median,Orientation_std,Dir_mean,Dir_median,Dir_std,NflId_mean,NflId_median,NflId_std,JerseyNumber_mean,JerseyNumber_median,JerseyNumber_std,YardLine_mean,YardLine_median,YardLine_std,Quarter_mean,Quarter_median,Quarter_std,Down_mean,Down_median,Down_std,Distance_mean,Distance_median,Distance_std,HomeScoreBeforePlay_mean,HomeScoreBeforePlay_median,HomeScoreBeforePlay_std,VisitorScoreBeforePlay_mean,VisitorScoreBeforePlay_median,VisitorScoreBeforePlay_std,DefendersInTheBox_mean,DefendersInTheBox_median,DefendersInTheBox_std,PlayerWeight_mean,PlayerWeight_median,PlayerWeight_std,Week_mean,Week_median,Week_std,Temperature_mean,Temperature_median,Temperature_std,Humidity_mean,Humidity_median,Humidity_std,X,Y,S,A,Dis,JerseyNumber,YardLine,Quarter,Down,Distance,HomeScoreBeforePlay,VisitorScoreBeforePlay,DefendersInTheBox,PlayerWeight,Week,Temperature,Humidity,GameClock_sec,PlayerHeight_dense,TimeDelta,PlayerAge,WindSpeed_dense,GameWeather_dense,IsRusher,IsRusherTeam,Orientation_sin,Orientation_cos,Dir_sin,Dir_cos,v_horizontal,v_vertical,diffScoreBeforePlay,OffenseDB,OffenseDL,OffenseLB,OffenseOL,OffenseQB,OffenseRB,OffenseTE,OffenseWR,DefenseDB,DefenseDL,DefenseLB,DefenseOL,time_quarter,time_end
0,0.551891,0.575753,0.168069,0.751212,0.8249,-0.293088,-1.271165,-1.27754,0.233781,-1.431361,-1.485642,-0.990188,-0.523003,-0.806444,2.108264,0.335753,-0.018753,0.841454,1.140839,0.500998,-0.873746,-0.80672,-1.333806,0.721812,-0.883239,-0.703052,-1.463273,0.50967,0.50967,0.0,-1.315124,-1.315124,0.0,2.191672,2.191672,0.0,-1.667696,-1.667696,0.0,-1.08475,-1.08475,0.0,-1.014376,-1.014376,0.0,-0.951534,-0.951534,0.0,-1.336098,-1.094662,0.680196,-1.618957,-1.618957,0.0,0.144426,0.144426,0.0,0.843111,0.843111,0.0,-0.513571,-0.327532,-1.352929,-0.976203,-1.851815,1.438089,-0.51517,-1.315124,2.191672,-1.667696,-1.08475,-1.014376,-0.951534,1.125669,-1.618957,0.144426,0.843111,1.477886,0.442041,-0.565651,-1.147190,0.16001,0.49451,-0.218218,-1.0,0.469544,-1.298784,1.395308,-0.133495,-0.616298,-1.293900,-0.154296,-0.027883,-0.099912,-0.034156,-0.268911,-0.072453,-0.465536,-0.657778,0.887027,2.111368,-1.865015,0.039706,-0.025452,1.477886,-1.633178
1,0.551891,0.575753,0.168069,0.751212,0.8249,-0.293088,-1.271165,-1.27754,0.233781,-1.431361,-1.485642,-0.990188,-0.523003,-0.806444,2.108264,0.335753,-0.018753,0.841454,1.140839,0.500998,-0.873746,-0.80672,-1.333806,0.721812,-0.883239,-0.703052,-1.463273,0.50967,0.50967,0.0,-1.315124,-1.315124,0.0,2.191672,2.191672,0.0,-1.667696,-1.667696,0.0,-1.08475,-1.08475,0.0,-1.014376,-1.014376,0.0,-0.951534,-0.951534,0.0,-1.336098,-1.094662,0.680196,-1.618957,-1.618957,0.0,0.144426,0.144426,0.0,0.843111,0.843111,0.0,-0.534034,-0.844261,-1.569755,-0.341225,-1.851815,1.508563,-0.51517,-1.315124,2.191672,-1.667696,-1.08475,-1.014376,-0.951534,0.712167,-1.618957,0.144426,0.843111,1.477886,0.424133,-0.565651,0.465623,0.16001,0.49451,-0.218218,-1.0,-0.613330,-1.202748,0.364136,1.226581,-1.095134,-1.035930,-0.154296,-0.027883,-0.099912,-0.034156,-0.268911,-0.072453,-0.465536,-0.657778,0.887027,2.111368,-1.865015,0.039706,-0.025452,1.477886,-1.633178
2,0.551891,0.575753,0.168069,0.751212,0.8249,-0.293088,-1.271165,-1.27754,0.233781,-1.431361,-1.485642,-0.990188,-0.523003,-0.806444,2.108264,0.335753,-0.018753,0.841454,1.140839,0.500998,-0.873746,-0.80672,-1.333806,0.721812,-0.883239,-0.703052,-1.463273,0.50967,0.50967,0.0,-1.315124,-1.315124,0.0,2.191672,2.191672,0.0,-1.667696,-1.667696,0.0,-1.08475,-1.08475,0.0,-1.014376,-1.014376,0.0,-0.951534,-0.951534,0.0,-1.336098,-1.094662,0.680196,-1.618957,-1.618957,0.0,0.144426,0.144426,0.0,0.843111,0.843111,0.0,-0.407713,-0.161737,-1.569755,-1.170793,-1.783078,0.063850,-0.51517,-1.315124,2.191672,-1.667696,-1.08475,-1.014376,-0.951534,-0.176863,-1.618957,0.144426,0.843111,1.477886,0.424133,-0.565651,2.404181,0.16001,0.49451,-0.218218,-1.0,0.05

## Divide features into groups

In [23]:
## dense features for play
dense_game_features = train_dense.columns[train_dense[:22].std() == 0]
## dense features for each player
dense_player_features = train_dense.columns[train_dense[:22].std() != 0]
## categorical features for play
cat_game_features = train_cat.columns[train_cat[:22].std() == 0]
## categorical features for each player
cat_player_features = train_cat.columns[train_cat[:22].std() != 0]

In [24]:
train_dense_game = train_dense[dense_game_features].iloc[np.arange(0, len(train), 22)].reset_index(drop = True).values
train_dense_game = np.hstack([train_dense_game, train_dense[dense_player_features][train_dense["IsRusher"] > 0]]) ## with rusher player feature

train_dense_players = [train_dense[dense_player_features].iloc[np.arange(k, len(train), 22)].reset_index(drop = True) for k in range(22)]
train_dense_players = np.stack([t.values for t in train_dense_players]).transpose(1, 0, 2)

train_cat_game = train_cat[cat_game_features].iloc[np.arange(0, len(train), 22)].reset_index(drop = True).values
train_cat_game = np.hstack([train_cat_game, train_cat[cat_player_features][train_dense["IsRusher"] > 0]]) ## with rusher player feature

train_cat_players = [train_cat[cat_player_features].iloc[np.arange(k, len(train), 22)].reset_index(drop = True) for k in range(22)]
train_cat_players = np.stack([t.values for t in train_cat_players]).transpose(1, 0, 2)

In [25]:
def return_step(x):
    temp = np.zeros(199)
    temp[x + 99:] = 1
    return temp

train_y_raw = train["Yards"].iloc[np.arange(0, len(train), 22)].reset_index(drop = True)
train_y = np.vstack(train_y_raw.apply(return_step).values)

In [26]:
train_dense_game.shape, train_dense_players.shape, train_cat_game.shape, train_cat_players.shape, train_y.shape

((23171, 106), (23171, 22, 82), (23171, 33), (23171, 22, 10), (23171, 199))

## Model

In [27]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import regularizers
import tensorflow as tf

In [28]:
keras.backend.clear_session()
def crps(y_true, y_pred):
    loss = K.mean((K.cumsum(y_pred, axis = 1) - y_true)**2)
    return loss

def get_model(batch_size, epochs):
    ## model dense
    input_dense_game = keras.layers.Input(shape=(train_dense_game.shape[1],))
    x1 = keras.layers.Dense(32, activation="relu")(input_dense_game)
    x1 = keras.layers.Dropout(0.5)(x1)
    # x1 = keras.layers.Dropout(0.1)(x1)

    input_dense_players = keras.layers.Input(shape=(train_dense_players.shape[1],train_dense_players.shape[2]))
    x2 = keras.layers.Dense(32, activation="relu")(input_dense_players)
    x2 = keras.layers.Dropout(0.5)(x2)
    # x2 = keras.layers.Flatten()(x2)
    # x2 = keras.layers.Dropout(0.1)(x2)

    ## model categorical
    input_cat_game = keras.layers.Input(shape=(train_cat_game.shape[1], ))
    embedding = keras.layers.Embedding(num_classes, 8, embeddings_regularizer=regularizers.l2(1))

    x3 = embedding(input_cat_game)
    x3 = keras.layers.Flatten()(x3)
    x3 = keras.layers.Dense(8, activation="relu")(x3)
    x3 = keras.layers.Dropout(0.6)(x3)

    input_cat_players = keras.layers.Input(shape=(train_cat_players.shape[1], train_cat_players.shape[2]))
    x4 = embedding(input_cat_players)

    x4 = keras.layers.Reshape((int(x4.shape[1]), int(x4.shape[2]) * int(x4.shape[3])))(x4)
    x4 = keras.layers.Dense(16, activation="relu")(x4)
    x4 = keras.layers.Dropout(0.6)(x4)

    ### concat players
    x_concat_players = keras.layers.Concatenate()([x2,x4])
    x_concat_players = keras.layers.Dense(16, activation="relu")(x_concat_players)
    # x_concat_players = keras.layers.GlobalAveragePooling1D()(x_concat_players)

    ## flatten
    x2 = keras.layers.Flatten()(x2)
    x4 = keras.layers.Flatten()(x4)
    x_concat_players = keras.layers.Flatten()(x_concat_players)

    ### concat all
    x_concat = keras.layers.Concatenate()([x1,x3,x_concat_players] + [x2, x4])
    x_concats = []
    n_unit = 128
    decay_rate = 0.5
    for k in range(5):
        x_concat = keras.layers.Dense(n_unit, activation="relu")(x_concat)
        x_concats.append(x_concat)
        n_unit = int(n_unit * decay_rate)
    x_concat = keras.layers.Concatenate()(x_concats)
    x_concat = keras.layers.Dropout(0.5)(x_concat)

    ## concat
    x_concat = keras.layers.Concatenate()([x1,x3,x_concat_players,x_concat] + [x2, x4])
    out_soft = keras.layers.Dense(199, activation="softmax", name = "out_soft")(x_concat)
    out_reg = keras.layers.Dense(1, activation=None, name = "out_reg")(x_concat)
    model = keras.models.Model(inputs = [input_dense_game, input_dense_players, input_cat_game, input_cat_players],
                               outputs = [out_soft, out_reg])

    ## compile
    #er = EarlyStopping(patience=10, min_delta=1e-4, restore_best_weights=True, monitor='val_out_soft_loss')
    model.compile(loss=[crps, keras.losses.mae],
                  loss_weights=[1.0, 0.01],
                  #optimizer=RAdam(warmup_proportion=0.1, min_lr=1e-7, learning_rate=0.0009, decay = 1e-5))
                  optimizer=keras.optimizers.Adam(learning_rate=0.006, decay = 1e-5))

    ## train
    tr_x = [train_dense_game[tr_inds], train_dense_players[tr_inds], train_cat_game[tr_inds], train_cat_players[tr_inds]]
    tr_y = [train_y[tr_inds], train_y_raw[tr_inds]/100]
    val_x = [train_dense_game[val_inds], train_dense_players[val_inds], train_cat_game[val_inds], train_cat_players[val_inds]]
    val_y = [train_y[val_inds], train_y_raw[val_inds]/100]
    model.fit(tr_x,
              tr_y,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(val_x, val_y)
              #callbacks=[er]
             )
    loss = model.history.history["val_out_soft_loss"][-1]
    return model, loss


In [29]:
#Kfold = 5 (100) com stop com feature statistic std: 0.013188597001135349
#Kfold = 5 (100) com stop com feature statistic mean, median, std: 0.013209664449095725

In [30]:
from sklearn.model_selection import train_test_split, KFold
losses = []
models = []
for k in range(2):
    kfold = KFold(10, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_y)):
        print("-----------")
        print("-----------")
        model, loss = get_model(32, 100)
        models.append(model)
        print(k_fold, loss)
        losses.append(loss)
print("-------")
print(losses)
print(np.mean(losses))

-----------
-----------
Train on 18536 samples, validate on 4635 samples
Epoch 1/100
18536/18536 [==============================] - 7s 377us/step - loss: 0.1650 - out_soft_loss: 0.0145 - out_reg_loss: 0.1969 - val_loss: 0.0139 - val_out_soft_loss: 0.0133 - val_out_reg_loss: 0.0646
Epoch 2/100
18536/18536 [==============================] - 6s 328us/step - loss: 0.0145 - out_soft_loss: 0.0137 - out_reg_loss: 0.0767 - val_loss: 0.0136 - val_out_soft_loss: 0.0131 - val_out_reg_loss: 0.0483
Epoch 3/100
18536/18536 [==============================] - 6s 338us/step - loss: 0.0143 - out_soft_loss: 0.0136 - out_reg_loss: 0.0605 - val_loss: 0.0136 - val_out_soft_loss: 0.0130 - val_out_reg_loss: 0.0457
Epoch 4/100
18536/18536 [==============================] - 6s 339us/step - loss: 0.0142 - out_soft_loss: 0.0136 - out_reg_loss: 0.0573 - val_loss: 0.0136 - val_out_soft_loss: 0.0130 - val_out_reg_loss: 0.0479
Epoch 5/100
18536/18536 [==============================] - 6s 336us/step - loss: 0.0142 - o

18537/18537 [==============================] - 7s 363us/step - loss: 0.0140 - out_soft_loss: 0.0134 - out_reg_loss: 0.0463 - val_loss: 0.0135 - val_out_soft_loss: 0.0130 - val_out_reg_loss: 0.0414
Epoch 23/100
18537/18537 [==============================] - 7s 357us/step - loss: 0.0140 - out_soft_loss: 0.0134 - out_reg_loss: 0.0457 - val_loss: 0.0136 - val_out_soft_loss: 0.0131 - val_out_reg_loss: 0.0466
Epoch 24/100
18537/18537 [==============================] - 7s 364us/step - loss: 0.0140 - out_soft_loss: 0.0134 - out_reg_loss: 0.0447 - val_loss: 0.0135 - val_out_soft_loss: 0.0130 - val_out_reg_loss: 0.0386
Epoch 25/100
18537/18537 [==============================] - 7s 360us/step - loss: 0.0139 - out_soft_loss: 0.0133 - out_reg_loss: 0.0457 - val_loss: 0.0136 - val_out_soft_loss: 0.0131 - val_out_reg_loss: 0.0427
1 0.013052511028945446
-----------
-----------
Train on 18537 samples, validate on 4634 samples
Epoch 1/100
18537/18537 [==============================] - 7s 378us/step - lo

18537/18537 [==============================] - 7s 362us/step - loss: 0.0139 - out_soft_loss: 0.0133 - out_reg_loss: 0.0467 - val_loss: 0.0141 - val_out_soft_loss: 0.0135 - val_out_reg_loss: 0.0442
Epoch 13/100
18537/18537 [==============================] - 7s 364us/step - loss: 0.0139 - out_soft_loss: 0.0133 - out_reg_loss: 0.0488 - val_loss: 0.0142 - val_out_soft_loss: 0.0136 - val_out_reg_loss: 0.0405
Epoch 14/100
18537/18537 [==============================] - 7s 359us/step - loss: 0.0139 - out_soft_loss: 0.0133 - out_reg_loss: 0.0465 - val_loss: 0.0141 - val_out_soft_loss: 0.0136 - val_out_reg_loss: 0.0414
Epoch 15/100
18537/18537 [==============================] - 7s 359us/step - loss: 0.0139 - out_soft_loss: 0.0133 - out_reg_loss: 0.0477 - val_loss: 0.0141 - val_out_soft_loss: 0.0136 - val_out_reg_loss: 0.0396
Epoch 16/100
18537/18537 [==============================] - 7s 356us/step - loss: 0.0139 - out_soft_loss: 0.0133 - out_reg_loss: 0.0439 - val_loss: 0.0141 - val_out_soft_los

In [31]:
print(losses)
print(np.mean(losses))

[0.012968644499778748, 0.013052511028945446, 0.01316608116030693, 0.013503354042768478, 0.013357731513679028]
0.013209664449095725


## Prediction

In [ ]:
def make_pred(test, sample, env, model):
    dt = features_estatisticas(test)
    test = pd.merge(dt, test, on='PlayId', how='outer')
    test = preprocess(test)
    test = drop(test)

    ### categorical
    test_cat = test[cat_features]
    for col in (test_cat.columns):
        test_cat.loc[:,col] = test_cat[col].fillna("nan")
        test_cat.loc[:,col] = col + "__" + test_cat[col].astype(str)
        isnan = ~test_cat.loc[:,col].isin(categories)
        if np.sum(isnan) > 0:
#             print("------")
#             print("test have unseen label : col")
            if not ((col + "__nan") in categories):
#                 print("not nan in train : ", col)
                test_cat.loc[isnan,col] = most_appear_each_categories[col]
            else:
#                 print("nan seen in train : ", col)
                test_cat.loc[isnan,col] = col + "__nan"
    for col in (test_cat.columns):
        test_cat.loc[:, col] = le.transform(test_cat[col])

    ### dense
    test_dense = test[dense_features]
    for col in (test_dense.columns):
        test_dense.loc[:, col] = test_dense[col].fillna(medians[col])
        test_dense.loc[:, col] = sss[col].transform(test_dense[col].values[:,None])

    ### divide
    test_dense_players = [test_dense[dense_player_features].iloc[np.arange(k, len(test), 22)].reset_index(drop = True) for k in range(22)]
    test_dense_players = np.stack([t.values for t in test_dense_players]).transpose(1,0, 2)

    test_dense_game = test_dense[dense_game_features].iloc[np.arange(0, len(test), 22)].reset_index(drop = True).values
    test_dense_game = np.hstack([test_dense_game, test_dense[dense_player_features][test_dense["IsRusher"] > 0]])
    
    test_cat_players = [test_cat[cat_player_features].iloc[np.arange(k, len(test), 22)].reset_index(drop = True) for k in range(22)]
    test_cat_players = np.stack([t.values for t in test_cat_players]).transpose(1,0, 2)

    test_cat_game = test_cat[cat_game_features].iloc[np.arange(0, len(test), 22)].reset_index(drop = True).values
    test_cat_game = np.hstack([test_cat_game, test_cat[cat_player_features][test_dense["IsRusher"] > 0]])

    test_inp = [test_dense_game, test_dense_players, test_cat_game, test_cat_players]
    
    ## pred
    pred = 0
    for model in models:
        _pred = model.predict(test_inp)[0]
        _pred = np.cumsum(_pred, axis = 1)
        pred += _pred
    pred /= len(models)
    pred = np.clip(pred, 0, 1)
    env.predict(pd.DataFrame(data=pred,columns=sample.columns))
    return pred

In [ ]:
env = nflrush.make_env()
preds = []
for test, sample in tqdm_notebook(env.iter_test()):
    pred = make_pred(test, sample, env, models)
    preds.append(pred)
env.write_submission_file()

In [ ]:
preds = np.vstack(preds)
## check whether prediction is submittable
print(np.mean(np.diff(preds, axis = 1) >= 0) == 1.0)
print(np.mean(preds > 1) == 0)

In [ ]:
print(losses)
print(np.mean(losses))